In [ ]:
import melign
import pandas as pd
dataset = melign.Dataset("dataset_test")
df = pd.read_excel("new_result.xlsx").sort_values(by="Alignment_Precision", ascending=True)
dataset = dataset[df["Sample"][:20]]

In [29]:
df.iloc[:20].describe()

,Unnamed: 0,Alignment_Precision,Alignment_Recall,Alignment_F1,Runtime_seconds
count,20.000000,20.000000,20.000000,20.000000,20.000000
mean,90.450000,0.843783,0.914863,0.875665,49.563669
std,50.672815,0.054081,0.067920,0.043155,25.471426
min,4.000000,0.695187,0.756264,0.780781,21.082742
25%,52.000000,0.821010,0.861771,0.855437,28.997110
50%,94.000000,0.867252,0.932246,0.879770,40.263539
75%,131.250000,0.879728,0.971683,0.910117,68.320290
max,176.000000,0.893617,0.991416,0.939980,104.101515


In [4]:
config = melign.get_default_config()
config.structural_align = True
config.candidate_min_length = 10
config

AlignConfig(score_model=<melign.align.score.MelodicsModel object at 0x000001D71FA3E210>, same_key=False, same_speed=True, speed_prior=1.0, variable_tail=True, local_tolerance=0.5, miss_tolerance=2, candidate_min_score=3.5, candidate_min_length=10, hop_length=1, split_melody=True, structural_align=True)

In [5]:
result_df, alignments = melign.eval_pipeline(dataset, config=config, save_plot=True, n_jobs=12)

Processing songs (n_jobs=12):   0%|          | 0/20 [00:00<?, ?it/s]

In [34]:
f1_scores = []
precisions = []
recalls = []
f1_old = []
precisions_old = []
recalls_old = []

for i, (name, alignment) in enumerate(alignments.items()):
    percentage = (len(alignment) + sum(m.sum_miss for m in alignment.matches)) / len(dataset[name].melody) * 100
    if percentage > 97.5:
        f1_scores.append(result_df["Alignment_F1"].iloc[i])
        precisions.append(result_df["Alignment_Precision"].iloc[i])
        recalls.append(result_df["Alignment_Recall"].iloc[i])
        f1_old.append(df["Alignment_F1"].iloc[i])
        precisions_old.append(df["Alignment_Precision"].iloc[i])
        recalls_old.append(df["Alignment_Recall"].iloc[i])
from statistics import mean
print(mean(f1_scores), mean(precisions), mean(recalls), len(f1_scores))
print(mean(f1_old), mean(precisions_old), mean(recalls_old), len(f1_old))

0.9561911954420603 0.9832326451806491 0.9316202290388863 11
0.8792463812445437 0.8293628321488594 0.9393357233684562 11


In [45]:
(mean(f1_scores) - mean(f1_old)) / 10, (mean(precisions) - mean(precisions_old)) / 20, (mean(recalls) - mean(recalls_old)) / 10

(np.float64(0.007694481419751664),
 np.float64(0.007693490651589485),
 np.float64(-0.0007715494329569861))

In [46]:
98.13 + 0.769*1.5

99.28349999999999